In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Prediction of number of jets

# Importing Libraries 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [ ]:
df = pd.read_csv('../input/multijet-primary-dataset/MultiJetRun2010B.csv')

# Exploratory data analysis

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

<h3>our data frame doesn't have null values</h3>

In [ ]:
df.describe()

# Data Visualization

In [ ]:
sns.set_style('whitegrid')
fig,ax = plt.subplots(nrows=3,ncols=3,figsize=(15,10))
ax = ax.flatten()
sns.scatterplot(ax=ax[0],x='MR',y='HT',data=df,hue='nJets')
sns.scatterplot(ax=ax[1],x='HT',y='MET',data=df,hue='nJets')
sns.scatterplot(ax=ax[2],x='MR',y='MET',data=df,hue='nJets')
sns.scatterplot(ax=ax[3],x='E1',y='E2',data=df,hue='nJets')
sns.scatterplot(ax=ax[4],x='E1',y='HT',data=df,hue='nJets')
sns.scatterplot(ax=ax[5],x='E2',y='HT',data=df,hue='nJets')
sns.scatterplot(ax=ax[6],x='E1',y='MR',data=df,hue='nJets')
sns.scatterplot(ax=ax[7],x='E2',y='MR',data=df,hue='nJets')
sns.scatterplot(ax=ax[8],x='Rsq',y='HT',data=df,hue='nJets')
plt.show()

In [ ]:
sns.set_style(style='whitegrid')
plt.figure(figsize=(25,10))
sns.heatmap(df.corr(),vmin=-1,vmax=1,annot=True,cmap='BuPu')
plt.show()

# Model Building

In [ ]:
y = df['nJets']
df = df.drop('nJets',axis=1)
df = df.drop('Event',axis=1)
train_x,test_x,train_y,test_y = train_test_split(df,y,test_size=0.2,random_state=42)
run = test_x['Run'].copy() # will be used as run column of solution data frame
drop = ['Run','nBJets','Lumi','MET','Rsq']
test_x = test_x.drop(drop,axis=1)
train_x = train_x.drop(drop,axis=1)
train_x

In [ ]:
model = RandomForestClassifier()

In [ ]:
model.fit(train_x,train_y)

In [ ]:
predict = model.predict(test_x)

In [ ]:
feature_importance = np.array(model.feature_importances_)
feature_names = np.array(train_x.columns)
data={'feature_names':feature_names,'feature_importance':feature_importance}
df_plt = pd.DataFrame(data)
df_plt.sort_values(by=['feature_importance'], ascending=False,inplace=True)
plt.figure(figsize=(10,8))
sns.barplot(x=df_plt['feature_importance'], y=df_plt['feature_names'])
plt.xlabel('FEATURE IMPORTANCE')
plt.ylabel('FEATURE NAMES')
plt.show()

In [ ]:
score = accuracy_score(test_y,predict)
print(score)

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as L

In [ ]:
input_shape=10

In [ ]:
train_y.unique()

In [ ]:
model = keras.Sequential()
model.add(L.Input(shape=input_shape))
model.add(L.Dense(128))
model.add(L.ReLU())
model.add(L.Dense(64))
model.add(L.ReLU())
model.add(L.Dense(8))
model.add(L.Softmax())

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),loss=keras.losses.SparseCategoricalCrossentropy(),metrics=keras.metrics.SparseCategoricalAccuracy())

In [ ]:
model.fit(x=train_x,y=train_y,epochs=30)

In [ ]:
preds = model.predict(test_x)

In [ ]:
prediction = np.argmax(preds,axis=1)

In [ ]:
accuracy_score(prediction,test_y)

In [ ]:
solution = pd.DataFrame({'Run':run,'nJets':prediction})
solution

In [ ]:
solution.to_csv('submission.csv',index=False)

<h1><center>Thank You</center></h1>